<a href="https://colab.research.google.com/github/basha-dev-coder/Machinelearning/blob/main/WandB_Sweeps_and_Bestmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 181 kB 46.8 MB/s 
     |████████████████████████████████| 145 kB 50.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=e66cf94cae88b7886a81f8c3b66fccb2bc8ece90f2aa9a99ca6b8f6f45572154
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [26]:
import wandb

sweep_config = {
  "name" : "my-sweep",
  "method" : "random",
  "parameters" : {
    "epochs" : {
      "values" : [30, 50, 70]
    },
    "learning_rate" :{
      "min": 0.0001,
      "max": 0.01
    }
  }
}

sweep_id = wandb.sweep(sweep_config)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: bb1bxai7
Sweep URL: https://wandb.ai/basha/uncategorized/sweeps/bb1bxai7


In [3]:
sweep_config["parameters"]["epochs"].get(0)

In [4]:
sweep_id

'vci8w3gj'

In [27]:
import tensorflow as tf
from tensorflow import keras
from wandb.keras import WandbCallback
from sklearn.metrics import roc_curve,f1_score,classification_report
tf.__version__

minst_data = keras.datasets.mnist
(X_train,y_train),(X_test,y_test) = minst_data.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [6]:
X_valid_norm,X_train_norm = X_train[:5000]/255.0 , X_train[5000:]/255.0
y_valid_norm,y_train_norm = y_train[:5000] , y_train[5000:]

In [8]:
def add_gaussian_noise(image):
    # image must be scaled in [0, 1]
    with tf.name_scope('Add_gaussian_noise'):
        noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=(50)/(255), dtype=tf.float32)
        noise_img = image + noise
        noise_img = tf.clip_by_value(noise_img, 0.0, 1.0)
    return noise_img

In [9]:

X_valid_norm_noise = X_valid_norm.copy()
for index,image in enumerate(X_valid_norm_noise):
  X_valid_norm_noise[index] = add_gaussian_noise(image)

In [10]:
def make_model(config):
  model_updating_lr = keras.Sequential([
        keras.layers.Flatten(input_shape=[28,28],name="input"),
        keras.layers.Dense(300,activation="relu",name="hidden_1"),
        keras.layers.Dense(100,activation="relu",name="hidden_2"),
        keras.layers.Dense(10,activation="softmax",name="output"),                  
  ])
  model_updating_lr.compile(optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate),metrics="accuracy",loss=keras.losses.SparseCategoricalCrossentropy())
  return model_updating_lr


In [11]:
# wandb.init(project="Sweep Experiments",group="minst_data") 

In [12]:
dummy_history = {}

In [13]:
def train():
    with wandb.init(project="Sweep Experiments",group="minst_Data") as run:
        config = wandb.config
        model = make_model(config)
        # print(config)
        # for epoch in range(config["epochs"]):
            # loss = model.fit()  # your model training code here
        # print("epoch ", epoch)
        history = model.fit(X_train_norm,y_train_norm,validation_data=(X_valid_norm_noise,y_valid_norm),batch_size=32,epochs=config["epochs"],callbacks=[WandbCallback()])
            # # print("loss ",loss.history)
            # wandb.log({"history":sum(loss.history["loss"])})
        wandb.log({"loss":sum(history.history["loss"]) / len(history.history["loss"]) ,
                   "accuracy":sum(history.history["accuracy"]) / len(history.history["accuracy"]),
                   "val_loss":sum(history.history["val_loss"]) / len(history.history["val_loss"]),
                   "val_accuracy":sum(history.history["val_accuracy"]) / len(history.history["val_accuracy"]),
                   "epochs": config["epochs"]})


count = 5 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

wandb: Agent Starting Run: z5roxz5j with config:
wandb: 	epochs: 30
wandb: 	learning_rate: 0.00897654188601601
wandb: Currently logged in as: basha. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/30
1719/1719 [==============================] - 10s 5ms/step - loss: 0.6367 - accuracy: 0.8371 - val_loss: 0.4237 - val_accuracy: 0.8888 - _timestamp: 1655802818.0000 - _runtime: 13.0000
Epoch 2/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3035 - accuracy: 0.9137 - val_loss: 0.3778 - val_accuracy: 0.8964 - _timestamp: 1655802823.0000 - _runtime: 18.0000
Epoch 3/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2508 - accuracy: 0.9290 - val_loss: 0.3214 - val_accuracy: 0.9110 - _timestamp: 1655802829.0000 - _runtime: 24.0000
Epoch 4/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2160 - accuracy: 0.9389 - val_loss: 0.2942 - val_accuracy: 0.9202 - _timestamp: 1655802834.0000 - _runtime: 29.0000
Epoch 5/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.1888 - accuracy: 0.9460 - val_loss: 0.3051 - val_accuracy: 0.9180 - _timestamp: 1655802840.0000 - _runtime: 35.0000
Epoch 6/30
1719/1719 [=======

accuracy,▁▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████████████▇
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epochs,▁
loss,█▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂
val_accuracy,▁▂▄▆▅▆▆▅▇▇▅█▆▅▄▄▄█▇▅▅▆▅▇▅▇█▅▃▆▅
val_loss,█▇▅▄▄▃▃▄▃▃▄▂▃▃▄▄▃▁▂▃▃▂▂▂▂▂▁▂▃▂▃
accuracy,0.96787
best_epoch,26
best_val_loss,0.19881
epoch,29
epochs,30


wandb: Agent Starting Run: tyhhjmit with config:
wandb: 	epochs: 50
wandb: 	learning_rate: 0.00701273718131376
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

Epoch 1/70
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7699 - accuracy: 0.8065 - val_loss: 0.4495 - val_accuracy: 0.8864 - _timestamp: 1655803539.0000 - _runtime: 7.0000
Epoch 2/70
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3328 - accuracy: 0.9053 - val_loss: 0.3716 - val_accuracy: 0.9006 - _timestamp: 1655803545.0000 - _runtime: 13.0000
Epoch 3/70
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2774 - accuracy: 0.9202 - val_loss: 0.3327 - val_accuracy: 0.9110 - _timestamp: 1655803550.0000 - _runtime: 18.0000
Epoch 4/70
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2440 - accuracy: 0.9301 - val_loss: 0.3213 - val_accuracy: 0.9138 - _timestamp: 1655803556.0000 - _runtime: 24.0000
Epoch 5/70
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2186 - accuracy: 0.9375 - val_loss: 0.3048 - val_accuracy: 0.9182 - _timestamp: 1655803561.0000 - _runtime: 29.0000
Epoch 6/70
1719/1719 [=========

accuracy,▁▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████████████████████▇
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epochs,▁
loss,█▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂
val_accuracy,▁▃▅▆▆▆▆▇▆█▆▆▆▆▆▇▆▆▆█▇▅▆▆▆▅▇▇▆▇▅▆▆▆▅▆▆▆▅▆
val_loss,█▆▄▄▄▄▃▃▃▂▃▂▂▃▂▂▂▂▂▁▂▂▂▂▂▃▂▁▂▁▂▂▂▂▂▂▁▂▂▂
accuracy,0.97962
best_epoch,34
best_val_loss,0.19558
epoch,69
epochs,70


wandb: Agent Starting Run: 9f0qdebt with config:
wandb: 	epochs: 30
wandb: 	learning_rate: 0.0018749902570955467
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 1/30
1719/1719 [==============================] - 6s 3ms/step - loss: 1.3448 - accuracy: 0.6815 - val_loss: 0.8328 - val_accuracy: 0.8348 - _timestamp: 1655803937.0000 - _runtime: 7.0000
Epoch 2/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5588 - accuracy: 0.8665 - val_loss: 0.5610 - val_accuracy: 0.8748 - _timestamp: 1655803943.0000 - _runtime: 13.0000
Epoch 3/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4217 - accuracy: 0.8882 - val_loss: 0.4739 - val_accuracy: 0.8854 - _timestamp: 1655803948.0000 - _runtime: 18.0000
Epoch 4/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3677 - accuracy: 0.8985 - val_loss: 0.4538 - val_accuracy: 0.8860 - _timestamp: 1655803953.0000 - _runtime: 23.0000
Epoch 5/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3365 - accuracy: 0.9053 - val_loss: 0.4173 - val_accuracy: 0.8928 - _timestamp: 1655803959.0000 - _runtime: 29.0000
Epoch 6/30
1719/1719 [=========

accuracy,▁▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████████████▇
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epochs,▁
loss,█▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂
val_accuracy,▁▄▅▅▆▆▇▆▇▇▆▇▇▇▇▇█▇██▇█▇███▇███▇
val_loss,█▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂
accuracy,0.92479
best_epoch,29
best_val_loss,0.3019
epoch,29
epochs,30


In [14]:
wandb.finish()

In [15]:

# wandb.init()
# config = wandb.config
# model = make_model(config)

In [16]:
# history_loss = model.fit(X_train_norm,y_train_norm,validation_data=(X_valid_norm_noise,y_valid_norm),batch_size=32,epochs=2,callbacks=[WandbCallback()])

In [17]:
# sum(history_loss.history["loss"])

In [18]:
# wandb.log({"loss": (sum(history_loss.history["loss"])) / len(history_loss.history["loss"]) , "epoch": config.epochs})

In [21]:
wandb.init()
learning_rate_efficient = 0.0089
model_best = keras.Sequential([
        keras.layers.Flatten(input_shape=[28,28],name="input"),
        keras.layers.Dense(300,activation="relu",name="hidden_1"),
        keras.layers.Dense(100,activation="relu",name="hidden_2"),
        keras.layers.Dense(10,activation="softmax",name="output"),                  
  ])
model_best.compile(optimizer=keras.optimizers.SGD(learning_rate=learning_rate_efficient),metrics="accuracy",loss=keras.losses.SparseCategoricalCrossentropy())

        # print(config)
        # for epoch in range(config["epochs"]):
            # loss = model.fit()  # your model training code here
        # print("epoch ", epoch)
history_best = model_best.fit(X_train_norm,y_train_norm,validation_data=(X_valid_norm_noise,y_valid_norm),batch_size=32,epochs=70,callbacks=[WandbCallback()])
        

Epoch 1/70
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6499 - accuracy: 0.8347 - val_loss: 0.3917 - val_accuracy: 0.9022 - _timestamp: 1655806238.0000 - _runtime: 8.0000
Epoch 2/70
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2956 - accuracy: 0.9170 - val_loss: 0.3353 - val_accuracy: 0.9156 - _timestamp: 1655806243.0000 - _runtime: 13.0000
Epoch 3/70
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2432 - accuracy: 0.9316 - val_loss: 0.2956 - val_accuracy: 0.9256 - _timestamp: 1655806248.0000 - _runtime: 18.0000
Epoch 4/70
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2093 - accuracy: 0.9405 - val_loss: 0.3054 - val_accuracy: 0.9192 - _timestamp: 1655806253.0000 - _runtime: 23.0000
Epoch 5/70
1719/1719 [==============================] - 5s 3ms/step - loss: 0.1837 - accuracy: 0.9477 - val_loss: 0.2722 - val_accuracy: 0.9324 - _timestamp: 1655806259.0000 - _runtime: 29.0000
Epoch 6/70
1719/1719 [=========

In [41]:
model_best.evaluate(X_train,y_train)

1875/1875 [==============================] - 5s 2ms/step - loss: 2.2915 - accuracy: 0.9944


[2.2915163040161133, 0.994350016117096]

In [42]:
predict_whole_data =  model_best.predict(X_train)
predict_whole_data_class = tf.argmax(predict_whole_data,1)

In [44]:
from sklearn.metrics import confusion_matrix
confusion_matrix(predict_whole_data_class,y_train)

array([[5920,    0,    0,    0,    0,    0,    4,    0,   16,    5],
       [   1, 6735,    5,    0,    2,    1,    5,    4,   61,    7],
       [   0,    0, 5947,    2,    2,    1,    1,    2,   29,    1],
       [   2,    3,    4, 6123,    1,   11,    0,    1,   84,   16],
       [   0,    2,    1,    0, 5833,    0,    1,    1,    2,    6],
       [   0,    0,    0,    2,    0, 5403,    0,    1,    5,    0],
       [   0,    1,    0,    0,    2,    2, 5907,    0,    5,    1],
       [   0,    1,    1,    3,    1,    1,    0, 6254,    4,   16],
       [   0,    0,    0,    0,    0,    1,    0,    0, 5642,    0],
       [   0,    0,    0,    1,    1,    1,    0,    2,    3, 5897]])

In [45]:
model_best.evaluate(X_valid_norm,y_valid_norm)

157/157 [==============================] - 0s 2ms/step - loss: 0.0700 - accuracy: 0.9816


[0.07001801580190659, 0.9815999865531921]

In [46]:
predict_without_noise_data =  model_best.predict(X_valid_norm)
predict_without_noise_data_class = tf.argmax(predict_without_noise_data,1)

In [48]:
confusion_matrix(predict_without_noise_data_class,y_valid_norm)

array([[473,   0,   0,   0,   0,   0,   1,   0,   1,   1],
       [  0, 557,   5,   0,   0,   1,   1,   0,   3,   1],
       [  0,   1, 475,   2,   1,   1,   1,   2,   0,   1],
       [  0,   2,   3, 480,   1,   3,   0,   0,   2,   4],
       [  0,   0,   1,   0, 530,   0,   1,   1,   0,   2],
       [  2,   0,   0,   4,   0, 425,   1,   1,   3,   0],
       [  1,   1,   1,   0,   2,   2, 495,   0,   1,   1],
       [  2,   0,   1,   3,   0,   1,   0, 543,   0,   5],
       [  1,   1,   2,   2,   0,   1,   1,   0, 451,   1],
       [  0,   1,   0,   2,   1,   0,   0,   3,   1, 479]])

In [93]:
model_best.evaluate(X_valid_norm,y_valid_norm)

157/157 [==============================] - 1s 4ms/step - loss: 0.0700 - accuracy: 0.9816


[0.07001801580190659, 0.9815999865531921]

In [50]:
predict_noise_data = model_best.predict(X_valid_norm_noise)

In [51]:
predict_noise_class = tf.argmax(predict_noise_data,1)

In [52]:
print(predict_noise_class == y_valid_norm)

tf.Tensor([ True  True  True ...  True False  True], shape=(5000,), dtype=bool)


In [53]:
 confusion_matrix(predict_noise_class,y_valid_norm)
 #from this matrix we can come to the conclusion that model is failed to predict Number 8 class, either we need more data or create some complex model to predict it
 #this confusion matrix is based on noise data, w/o noise it is able to predict most of the classes correctly

array([[469,   0,   1,   0,   0,   0,   1,   1,   0,   1],
       [  0, 423,   2,   0,   1,   0,   0,   0,   1,   1],
       [  0,   6, 461,   3,   4,   2,   2,   6,   0,   1],
       [  1,   8,   4, 469,   1,   5,   0,  10,   2,   8],
       [  0,   0,   1,   0, 501,   0,   2,   1,   0,   4],
       [  2,   0,   1,   6,   0, 420,   7,   2,   2,   2],
       [  1,   0,   0,   0,   2,   1, 453,   0,   0,   0],
       [  0,   0,   1,   1,   1,   0,   0, 504,   0,   4],
       [  6, 126,  17,  13,  11,   6,  36,  22, 457,  44],
       [  0,   0,   0,   1,  14,   0,   0,   4,   0, 430]])

In [58]:
model_best.save("best_model.h5")

In [59]:
wandb.save("best_model.h5")

['/content/wandb/run-20220621_101029-9f0qdebt/files/best_model.h5']

In [6]:
wanbd_best = wandb.restore("model-best.h5",run_path="basha/uncategorized/9f0qdebt")

In [11]:
wanbd_best = keras.models.load_model(wanbd_best.name)

In [81]:
wanbd_best.evaluate(X_train,y_train)

1875/1875 [==============================] - 7s 4ms/step - loss: 4.2351 - accuracy: 0.9876


[4.235066890716553, 0.987583339214325]

In [82]:
wandb_predict_noise_data = wanbd_best.predict(X_valid_norm_noise)
wandb_predict_noise_class = tf.argmax(wandb_predict_noise_data,1)

In [83]:
confusion_matrix(wandb_predict_noise_class,y_valid_norm)

array([[472,   0,   1,   0,   0,   0,   1,   3,   0,   2],
       [  0, 495,   2,   0,   1,   0,   1,   0,   1,   1],
       [  0,   7, 460,   1,   4,   2,   3,   5,   0,   1],
       [  1,   3,   5, 474,   1,   6,   0,  13,   2,   8],
       [  0,   0,   1,   0, 510,   1,   3,   1,   0,   4],
       [  2,   0,   1,   6,   0, 419,   6,   2,   2,   1],
       [  1,   0,   0,   0,   2,   1, 465,   0,   0,   0],
       [  0,   0,   2,   1,   1,   0,   0, 507,   0,   5],
       [  3,  58,  16,   9,   7,   5,  22,  13, 457,  30],
       [  0,   0,   0,   2,   9,   0,   0,   6,   0, 443]])

In [84]:
wanbd_best.evaluate(X_valid_norm_noise,y_valid_norm)

157/157 [==============================] - 1s 6ms/step - loss: 0.1759 - accuracy: 0.9404


[0.17590932548046112, 0.9404000043869019]

In [86]:
wandb_predict_without_noise_data = wanbd_best.predict(X_valid_norm)
wandb_predict_without_noise_class = tf.argmax(wandb_predict_without_noise_data,1)

In [88]:
confusion_matrix(wandb_predict_without_noise_class,y_valid_norm)

array([[473,   0,   0,   0,   0,   0,   1,   1,   1,   3],
       [  0, 556,   6,   0,   1,   1,   1,   2,   4,   1],
       [  0,   2, 472,   3,   2,   1,   0,   2,   0,   0],
       [  1,   2,   3, 482,   1,   3,   0,   0,   2,   3],
       [  1,   0,   1,   0, 529,   0,   1,   1,   1,   5],
       [  1,   0,   0,   3,   0, 424,   0,   1,   3,   0],
       [  1,   1,   1,   0,   1,   2, 497,   0,   1,   1],
       [  2,   0,   2,   2,   0,   1,   0, 539,   1,   5],
       [  0,   1,   3,   1,   0,   1,   1,   0, 449,   0],
       [  0,   1,   0,   2,   1,   1,   0,   4,   0, 477]])

In [89]:
wanbd_best.evaluate(X_valid_norm,y_valid_norm)

157/157 [==============================] - 1s 4ms/step - loss: 0.0690 - accuracy: 0.9796


[0.06896845996379852, 0.9796000123023987]

In [12]:
wandb_predict_whole_data = wanbd_best.predict(X_train)
wandb_predict_whole_class = tf.argmax(wandb_predict_whole_data,1)

In [13]:
confusion_matrix(wandb_predict_whole_class,y_train)

NameError: ignored

In [92]:
wanbd_best.evaluate(X_train,y_train)

1875/1875 [==============================] - 4s 2ms/step - loss: 4.2351 - accuracy: 0.9876


[4.235066890716553, 0.987583339214325]

In [27]:
for _ in ["macro","micro","weighted",None]:
  # print(_)
  print(f1_score(wandb_predict_whole_class,y_train,average=_))

0.9875440286760708
0.9875833333333334
0.9876098732982805
[0.99327731 0.98735666 0.990778   0.98089274 0.99122882 0.98762215
 0.99215521 0.99232859 0.97416134 0.98563947]


In [45]:
classnames = ["0","1","2","3","4","5","6","7","8","9"]
cls_report = classification_report(wandb_predict_whole_class,y_train,target_names= classnames)

In [46]:
print(cls_report)

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5977
           1       1.00      0.98      0.99      6862
           2       0.99      0.99      0.99      5970
           3       1.00      0.97      0.98      6325
           4       1.00      0.99      0.99      5901
           5       0.98      1.00      0.99      5324
           6       0.99      0.99      0.99      5937
           7       0.99      0.99      0.99      6249
           8       0.95      1.00      0.97      5566
           9       0.98      0.99      0.99      5889

    accuracy                           0.99     60000
   macro avg       0.99      0.99      0.99     60000
weighted avg       0.99      0.99      0.99     60000



In [47]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [211]:
import random
my_num = random.sample(range(-100,100),100)
my_output = [x*3+4 for x in my_num]

my_num_valid,my_output_valid = my_num[:25] , my_output[:25]
my_num_train,my_output_train = my_num[25:] , my_output[25:] 
# relu = [0 if x < 0 else x for x in my_num]

In [51]:
len(my_num_valid) , len(my_num_train)

(25, 75)

In [212]:
my_num_train,my_output_train = tf.constant(my_num_train) , tf.constant(my_output_train)
my_num_valid,my_output_valid = tf.constant(my_num_valid) , tf.constant(my_output_valid)

In [53]:
my_num_train.shape,my_output_train.shape

(TensorShape([75]), TensorShape([75]))

In [120]:
my_num_valid.shape

TensorShape([25])

In [213]:
my_num_train,my_output_train = tf.reshape(my_num_train,[1,-1]) , tf.reshape(my_output_train,[1,-1])
my_num_valid,my_output_valid = tf.reshape(my_num_valid,[1,-1]) , tf.reshape(my_output_valid,[1,-1])

In [ ]:
tf.reshape(my_output_valid,[1,-1])

In [141]:
my_output_valid.shape

TensorShape([25])

In [ ]:
my_output_valid

In [196]:
my_num_train

<tf.Tensor: shape=(1, 75), dtype=int32, numpy=
array([[-17, -91, -52, -45,  10,  73,  28, -98, -41, -51,  30,  70,  98,
        -27, -96, -46, -37,  65, -72,   2, -50,  -5, -24,  32,  44,   4,
        -26,  91,  43,  97, -29, -88,  57, -59, -74,  38, -99, -15,  21,
         94,  34, -81, -93,  31,  29,  -3,  -2, -73,  49,   6,  85, -68,
         11, -71,  14, -14,  35,  20,  51,  56, -40, -10, -56,  92,  -6,
        -95,  13, -28,  93, -60,  95, -85, -54,  59,  88]], dtype=int32)>

In [8]:
alpha = 0.01
leaky_relu = [alpha*x if x<0 else x for x in my_num]

In [9]:
leaky_relu

[10, 40, 20, 30, 40, 50, 1, 3, 65, 1000, -0.02, -0.04, -0.1]

In [128]:
my_num_train.shape[1:]

TensorShape([75])

In [214]:
model = keras.models.Sequential([
       keras.layers.Dense(75,input_shape=my_num_train.shape[1:]),
       keras.layers.Dense(50,activation="relu"),
       keras.layers.Dense(1)                          
])
model.compile(optimizer=keras.optimizers.SGD(),loss="mse",metrics="mse")

In [152]:
# model.build(input_shape=(1,75))
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_64 (Dense)            (None, 75)                5700      
                                                                 
 dense_65 (Dense)            (None, 1)                 76        
                                                                 
Total params: 5,776
Trainable params: 5,776
Non-trainable params: 0
_________________________________________________________________


In [215]:
my_num_train

<tf.Tensor: shape=(1, 75), dtype=int32, numpy=
array([[ -46,  -31,  -71,  -66,  -97,  -76,    8,  -19,   42,   22,   77,
          10,  -29,  -72,   47,   40,  -53,   19,  -74,    2,  -13,  -67,
          36,   66,  -43,  -94,   33,  -93,   92,   99,  -68,   95,   64,
         -69,   90,    4,   59,   97,   -5,   80,   69,  -42,   54,  -14,
           7,  -57,   16,   93,  -92,  -64,  -38,  -21,  -84,   96,  -75,
          67,   53, -100,  -77,   32,   38,   48,   57,  -34,  -82,  -63,
          11,   26,   88,   24,  -47,   79,   83,   49,   44]],
      dtype=int32)>

In [216]:
tf.keras.activations.selu(
   tf.cast(my_num_train,dtype=tf.float16)
)

<tf.Tensor: shape=(1, 75), dtype=float16, numpy=
array([[ -1.758,  -1.758,  -1.758,  -1.758,  -1.758,  -1.758,   8.41 ,
         -1.758,  44.12 ,  23.12 ,  80.94 ,  10.51 ,  -1.758,  -1.758,
         49.38 ,  42.03 ,  -1.758,  19.97 ,  -1.758,   2.102,  -1.758,
         -1.758,  37.8  ,  69.4  ,  -1.758,  -1.758,  34.7  ,  -1.758,
         96.7  , 104.   ,  -1.758,  99.8  ,  67.25 ,  -1.758,  94.56 ,
          4.203,  62.   , 101.94 ,  -1.746,  84.06 ,  72.5  ,  -1.758,
         56.75 ,  -1.758,   7.355,  -1.758,  16.81 ,  97.75 ,  -1.758,
         -1.758,  -1.758,  -1.758,  -1.758, 100.9  ,  -1.758,  70.4  ,
         55.7  ,  -1.758,  -1.758,  33.62 ,  39.94 ,  50.44 ,  59.9  ,
         -1.758,  -1.758,  -1.758,  11.56 ,  27.31 ,  92.5  ,  25.22 ,
         -1.758,  83.   ,  87.2  ,  51.5  ,  46.25 ]], dtype=float16)>

In [217]:
tf.keras.activations.elu(
    tf.cast(my_num_train,dtype=tf.float16), alpha=1.0
)

<tf.Tensor: shape=(1, 75), dtype=float16, numpy=
array([[-1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,  8.   , -1.   ,
        42.   , 22.   , 77.   , 10.   , -1.   , -1.   , 47.   , 40.   ,
        -1.   , 19.   , -1.   ,  2.   , -1.   , -1.   , 36.   , 66.   ,
        -1.   , -1.   , 33.   , -1.   , 92.   , 99.   , -1.   , 95.   ,
        64.   , -1.   , 90.   ,  4.   , 59.   , 97.   , -0.993, 80.   ,
        69.   , -1.   , 54.   , -1.   ,  7.   , -1.   , 16.   , 93.   ,
        -1.   , -1.   , -1.   , -1.   , -1.   , 96.   , -1.   , 67.   ,
        53.   , -1.   , -1.   , 32.   , 38.   , 48.   , 57.   , -1.   ,
        -1.   , -1.   , 11.   , 26.   , 88.   , 24.   , -1.   , 79.   ,
        83.   , 49.   , 44.   ]], dtype=float16)>

In [235]:
tf.nn.leaky_relu(
     tf.cast(my_num_train,dtype=tf.float16), alpha=0.2, name=None
)

<tf.Tensor: shape=(1, 75), dtype=float16, numpy=
array([[ -9.195,  -6.2  , -14.195, -13.195, -19.39 , -15.195,   8.   ,
         -3.799,  42.   ,  22.   ,  77.   ,  10.   ,  -5.797, -14.4  ,
         47.   ,  40.   , -10.59 ,  19.   , -14.8  ,   2.   ,  -2.6  ,
        -13.4  ,  36.   ,  66.   ,  -8.6  , -18.8  ,  33.   , -18.6  ,
         92.   ,  99.   , -13.59 ,  95.   ,  64.   , -13.8  ,  90.   ,
          4.   ,  59.   ,  97.   ,  -1.   ,  80.   ,  69.   ,  -8.4  ,
         54.   ,  -2.799,   7.   , -11.4  ,  16.   ,  93.   , -18.39 ,
        -12.8  ,  -7.598,  -4.2  , -16.8  ,  96.   , -15.   ,  67.   ,
         53.   , -20.   , -15.4  ,  32.   ,  38.   ,  48.   ,  57.   ,
         -6.797, -16.39 , -12.59 ,  11.   ,  26.   ,  88.   ,  24.   ,
         -9.4  ,  79.   ,  83.   ,  49.   ,  44.   ]], dtype=float16)>

In [200]:
my_num_train_scale

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [220]:
scaler.inverse_transform(my_num_train_scale)

array([[ -46.,  -31.,  -71.,  -66.,  -97.,  -76.,    8.,  -19.,   42.,
          22.,   77.,   10.,  -29.,  -72.,   47.,   40.,  -53.,   19.,
         -74.,    2.,  -13.,  -67.,   36.,   66.,  -43.,  -94.,   33.,
         -93.,   92.,   99.,  -68.,   95.,   64.,  -69.,   90.,    4.,
          59.,   97.,   -5.,   80.,   69.,  -42.,   54.,  -14.,    7.,
         -57.,   16.,   93.,  -92.,  -64.,  -38.,  -21.,  -84.,   96.,
         -75.,   67.,   53., -100.,  -77.,   32.,   38.,   48.,   57.,
         -34.,  -82.,  -63.,   11.,   26.,   88.,   24.,  -47.,   79.,
          83.,   49.,   44.]])

In [233]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
my_num_train_scale = scaler.fit_transform(my_num_train)
my_output_train_scale = scaler.transform(my_output_train)

In [223]:
my_num_valid_scale = scaler.transform(my_num_valid)

ValueError: ignored

In [234]:
history = model.fit(my_num_train_scale,my_output_train_scale,epochs=10,verbose=1)

Epoch 1/10
1/1 [==============================] - 0s 486ms/step - loss: 15184.6396 - mse: 15184.6396
Epoch 2/10
1/1 [==============================] - 0s 10ms/step - loss: 15182.2715 - mse: 15182.2715
Epoch 3/10
1/1 [==============================] - 0s 9ms/step - loss: 15179.9980 - mse: 15179.9980
Epoch 4/10
1/1 [==============================] - 0s 9ms/step - loss: 15177.8135 - mse: 15177.8135
Epoch 5/10
1/1 [==============================] - 0s 10ms/step - loss: 15175.7148 - mse: 15175.7148
Epoch 6/10
1/1 [==============================] - 0s 13ms/step - loss: 15173.7002 - mse: 15173.7002
Epoch 7/10
1/1 [==============================] - 0s 10ms/step - loss: 15171.7646 - mse: 15171.7646
Epoch 8/10
1/1 [==============================] - 0s 17ms/step - loss: 15169.9053 - mse: 15169.9053
Epoch 9/10
1/1 [==============================] - 0s 8ms/step - loss: 15168.1201 - mse: 15168.1201
Epoch 10/10
1/1 [==============================] - 0s 14ms/step - loss: 15166.4062 - mse: 15166.4062


In [76]:
tf.reshape(my_num_train,[1,-1]).shape

TensorShape([1, 75])

In [68]:
my_num_train

<tf.Tensor: shape=(75,), dtype=int32, numpy=
array([-59,  33,  49,  37,  44, -33, -48,  -4, -81, -14,  18,  66, -13,
       -29, -38, -26, -73,  63,  57,  89, -44, -74,  21, -16,   3, -64,
        85, -99,  -3, -10,  43,  -9, -17,  12,  40,  52, -34, -60, -15,
        90, -55, -84,  80,  10, -53,  13, -45, -36, -90, -68, -49, -89,
       -12, -40, -25,  41,  74,  71, -93, -61,  16, -62, -52, -57,  17,
         9,  72,   1, -19, -87,  46, -30, -76, -85,  27], dtype=int32)>